In [1]:
import csv
from pprint import pprint
from collections import defaultdict
import pandas as pd 
import numpy as np
from sklearn import linear_model
from sklearn import svm

In [2]:
df = pd.read_table("anime.csv", sep=",")
us = pd.read_table("rating.csv", sep=",")

In [3]:
animeList={} #Anime and their data
genreList=defaultdict(list) #Animes sorted by genre
userList=defaultdict(list) #User's list of anime and rating in a (anime_id,rating) structure
userRating=defaultdict(list) #User's rating
userCategory=defaultdict(set) #User's list of categories they have watched

In [4]:
# To Get data from animeList. dictionary = animeList['id']['data']
def getData(dictionary):
    if not dictionary.isnull().values[0]:
        return dictionary.values[0]
    else:
        return 0

In [5]:
def getCategory(data):
    cat=[]
    if data:
        for x in data.split(', '):
            cat.append(x)
    return cat

In [6]:
for index, row in df.iterrows():
    animeList[row['anime_id']]=df.loc[[index]]
    if not df.loc[[index]]['genre'].isnull().values[0]:
        for x in row['genre'].split(', '):
            genreList[x].append(row['anime_id'])

In [ ]:
for index, row in us.iterrows():
    userList[row['user_id']].append(row['anime_id'])
    userRating[row['user_id']].append(row['rating'])

In [ ]:
for index, row in us.iterrows():
    if int(row['anime_id']) in animeList:
        catList= getCategory(getData(animeList[int(row['anime_id'])]['genre']))
        for x in catList:
            userCategory[row['user_id']].add(x)
    

In [ ]:
#Average rating per user including -1 for no review (KEEP or remove -1???)
averageUser=defaultdict(float)
stdUser=defaultdict(float)
for key in userRating:
    averageUser[key]=np.mean(userRating[key])
    stdUser[key]=np.std(userRating[key])

In [ ]:
genreOrder=sorted(genreList.keys())
feat=[]
for index, row in us.iterrows():
    temp=[]
    if int(row['anime_id']) in animeList:
        catList= getCategory(getData(animeList[int(row['anime_id'])]['genre']))
    for x in genreOrder:
        if x in catList:
            temp.append(1)
        else:
            temp.append(0)
    feat.append(temp)

In [ ]:
X_featureSet=feat
Y_featureSet=[row['rating'] for index,row in us.iterrows()]

In [ ]:
len(X_featureSet)/3

In [ ]:
# I gave up running this. It never finished.... I am not sure whether prediction rating by genere is a good idea. 
lam = [0.01,0.1,1,10]

def difference(pred,act):
    correct=0
    
    for i in range(0, len(pred)):
        if pred[i]==act[i]:
            correct = correct + 1
    return float(correct)/len(pred)
    
clf=svm.LinearSVC(C=1)
clf.fit(X_featureSet[:int(len(X_featureSet)/3)],Y_featureSet[:int(len(Y_featureSet)/3)])

valid_predictions = clf.predict(X_featureSet[int(len(X_featureSet)/3):int(2*len(X_featureSet)/3)])

print("lambda = " + str(l) + ":\taccuracy_valid=" + str(difference(valid_predictions,Y_featureSet[int(len(Y_featureSet)/3):int(2*len(Y_featureSet)/3)])))

In [8]:
from scipy.spatial.distance import cosine

In [18]:
#cosine similarity between animes
genereList=sorted(genreList.keys())

animeFeatureList=[]
for key in animeList:
    temp=[]
    catList= getCategory(getData(animeList[key]['genre']))
    for x in genreList:
        if x in catList:
            temp.append(1)
        else:
            temp.append(0)
    animeFeatureList.append((key,temp))
    

In [30]:
# this taks a while too, but it seems to work as intended i think.
cosine_anime=defaultdict(float)
for i in range(0,len(animeFeatureList)):
    for j in range(i,len(animeFeatureList)):
        cosine_anime[animeFeatureList[i][0],animeFeatureList[j][0]]=1-cosine(animeFeatureList[i][1],animeFeatureList[j][1])

C:\Users\Charles\Anaconda3\lib\site-packages\scipy\spatial\distance.py:505: RuntimeWarning: invalid value encountered in true_divide
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


KeyboardInterrupt: 

In [31]:
cosine_anime

defaultdict(float,
            {(32281, 32281): 1.0,
             (32281, 5114): 0.18898223650461365,
             (32281, 28977): 0.0,
             (32281, 9253): 0.0,
             (32281, 9969): 0.0,
             (32281, 32935): 0.44721359549995787,
             (32281, 11061): 0.0,
             (32281, 820): 0.25,
             (32281, 15335): 0.0,
             (32281, 15417): 0.0,
             (32281, 4181): 0.67082039324993692,
             (32281, 28851): 0.57735026918962584,
             (32281, 918): 0.0,
             (32281, 2904): 0.20412414523193156,
             (32281, 28891): 0.44721359549995787,
             (32281, 199): 0.57735026918962584,
             (32281, 23273): 0.67082039324993692,
             (32281, 24701): 0.18898223650461365,
             (32281, 12355): 0.0,
             (32281, 1575): 0.20412414523193156,
             (32281, 263): 0.25,
             (32281, 44): 0.40824829046386313,
             (32281, 1): 0.20412414523193156,
             (32281, 30276

http://jill-jenn.net/_static/works/mangaki-recsys2015.pdf 